In [49]:
# 简单实现model.fit()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [50]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [51]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [53]:
# metric 算子 评价指标 评价函数
metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print('-' * 50)
print(metric([0.], [1.]))
print('-' * 50)
# 具有累加功能 (9+1)/2=5
print(metric.result())
print('-' * 50)
# 清空累加 重置reset_states
metric.reset_states()  # 每次epoch需要reset_states
metric([1.], [3.])
print(metric.result())
metric([1.], [2.])
print(metric.result())
print('-' * 50)

tf.Tensor(9.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(5.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(5.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(2.5, shape=(), dtype=float32)
--------------------------------------------------


In [54]:
# 训练集的样本数
print(len(x_train_scaled))
# 特征数
print(x_train.shape[1:])
# 每次训练给予的样本数 batch_size默认32
print(len(x_train_scaled) / 32)

11610
(8,)
362.8125


In [55]:
t = np.arange(6).reshape(1, 2, 1, 3)
print(t)
print('-' * 50)
print(tf.squeeze(t))  # 降维 从形状中删除单维度条目，即把shape中为1的维度去掉
print('-' * 50)
# squeeze的作用
t = tf.constant([[1], [2], [3]])
print(t)
print('-' * 50)
tf.squeeze(t, 1)

[[[[0 1 2]]

  [[3 4 5]]]]
--------------------------------------------------
tf.Tensor(
[[0 1 2]
 [3 4 5]], shape=(2, 3), dtype=int32)
--------------------------------------------------
tf.Tensor(
[[1]
 [2]
 [3]], shape=(3, 1), dtype=int32)
--------------------------------------------------


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

In [56]:
idx = np.random.randint(0, 1000, size=5)
print(idx)
print(x_train_scaled[idx])
print(y_train[idx])  # 随机挑选的5个样本的标签值

[451 689 123 239 671]
[[-1.15189191 -0.68904142 -0.71832594 -0.0344517   0.04849902  0.02079551
  -1.35026948  1.24085054]
 [-0.02243588  0.75276284 -0.02553397 -0.07790702  0.10112162  0.0877423
  -0.76239258  0.75149068]
 [ 1.20351706 -0.28854023  0.2885874  -0.37900997 -0.27903129  0.00491105
  -0.68307586  0.31206551]
 [ 0.24030638 -1.89054496 -0.21586094 -0.26176729 -0.1012032  -0.17313388
  -0.67841017  0.28709817]
 [ 0.17199659  1.07316379 -0.69217902 -0.22122628 -0.13205093  0.15867472
  -0.79038672  0.60668011]]
[0.955 1.71  2.549 2.279 1.639]


In [57]:
epochs = 100
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size
print(steps_per_epoch)
print('-' * 50)
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()


# 随机取出来32个样本
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]


model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])

362
--------------------------------------------------


In [58]:
print(model.variables)

[<tf.Variable 'dense_8/kernel:0' shape=(8, 30) dtype=float32, numpy=
array([[ 0.3803678 , -0.37519726,  0.17287979, -0.05467156,  0.22848412,
         0.09458029, -0.3810946 ,  0.16130951,  0.26306012,  0.29926738,
         0.3465807 ,  0.38341567,  0.24388853, -0.11799154, -0.28580767,
         0.13076338, -0.27784705,  0.3406634 ,  0.11337647,  0.09574112,
         0.29668066,  0.08994505,  0.13591579, -0.37105912,  0.24116084,
         0.17388257,  0.30370453, -0.04434636, -0.2885944 ,  0.21909162],
       [ 0.3883346 , -0.3758412 , -0.31106415,  0.2490237 ,  0.01044884,
        -0.16571085,  0.07318476,  0.08807188, -0.3312165 ,  0.06268877,
         0.20999983,  0.2725329 , -0.37625957,  0.21355662,  0.10077548,
        -0.28984636, -0.03849173, -0.19078694, -0.19603837, -0.32810873,
         0.040696  ,  0.22382161,  0.2241306 ,  0.18545559, -0.39208877,
        -0.14910027,  0.0320704 ,  0.30152366,  0.06975621, -0.29280862],
       [ 0.33188984,  0.22721389, -0.24318479,  0.072

In [59]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 30)                270       
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
# 代替model.fit()
for epoch in range(epochs):  # 每一轮epochs训练所有的样本
    metric.reset_states()  # 清空重置损失
    for step in range(steps_per_epoch):
        # 随机取32个样本
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            # 得到预测值 等价于model.predict(用于批量处理大量输入)
            y_pred = model(x_batch)  # 函数式调用(调用__call__方法) 适合小批量数据
            # model(x_valid_scaled)返回二维(num_examples, 1)，而y_pred是一维(num_examples, )
            # 二阶张量变为一阶张量
            y_pred = tf.squeeze(y_pred, 1)
            # 计算损失
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            # 一次epoch的均方误差
            metric(y_batch, y_pred)
        # 求梯度
        grads = tape.gradient(loss, model.variables)
        # 梯度和变量绑定
        grads_and_vars = zip(grads, model.variables)
        # 通过grads更新模型的model.variables 即w,b
        optimizer.apply_gradients(grads_and_vars)
        print("Epoch " + str(epoch) + " train mse:" + str(metric.result().numpy()), end='\r')
    print('')  # 新起一行显示
    # 验证集验证
    y_valid_pred = model(x_valid_scaled)
    # print(y_valid_pred.shape) # (3870, 1)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
    # print(y_valid_pred.shape) # (3870, )
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print('\t', 'valid mse', valid_loss.numpy())





Epoch 0 train mse:1.4474707
	 valid mse 0.5429420816535974
Epoch 1 train mse:0.47183568
	 valid mse 0.46821941101572345
Epoch 2 train mse:0.43330864
	 valid mse 0.43689425456394876
Epoch 3 train mse:0.41948608
	 valid mse 0.44954535944697277
Epoch 4 train mse:0.42072552
	 valid mse 0.4846302026112092
Epoch 5 train mse:0.38212462
	 valid mse 0.3930751755417499
Epoch 6 train mse:0.38140512
	 valid mse 0.3842931467330847
Epoch 7 train mse:0.36867523
	 valid mse 0.3789525180270421
Epoch 8 train mse:0.37161024
	 valid mse 0.3764392560064972
Epoch 9 train mse:0.36818838
	 valid mse 0.3723501177037111
Epoch 10 train mse:0.36316065
	 valid mse 0.45840851322212944
Epoch 11 train mse:0.35591495
	 valid mse 0.37091831357920124
Epoch 12 train mse:0.37033064
	 valid mse 0.7500539140960183
Epoch 13 train mse:0.36093804
	 valid mse 0.37151266102108216
Epoch 14 train mse:0.36973777
	 valid mse 0.3628605171736531
Epoch 15 train mse:0.34583664
	 valid mse 0.36244670029045195
Epoch 16 train mse:0.3430935